In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model

#inloggen bij de spotify API en playlist link
CLIENT_ID = '82c974c12dd94f52a53884b52640c3c0'
CLIENT_SECRET = "9ea825d8d4b04e9e822cb845cade61a0"
PLAYLIST_LINK = "https://open.spotify.com/playlist/3VJlwgnV4IaxGK8uIEZMjV?si=ca8c506dd5d04663"

CLIENT_CREDENTIALS_MANAGER = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)
SP = spotipy.Spotify(client_credentials_manager=CLIENT_CREDENTIALS_MANAGER)

#get playlist
def get_playlist_uri(playlist_link):
    return playlist_link.split("/")[-1].split("?")[0]

#nummers + gegevens uit de playlist halen
def get_tracks():
    tracks = []
    playlist_uri = get_playlist_uri(PLAYLIST_LINK)
    for track in SP.playlist_tracks(playlist_uri)["items"]:
        track_uri = track["track"]["uri"]
        track_name = track["track"]["name"]
        result = track_name, SP.audio_features(track_uri)
        tracks.append(result)

    return tracks

data = pd.read_excel("D:/Universiteit/OZP/spotify_genre_final.xlsx")
data
data = data.drop(["Album_cover_link", "release_date_precision", "release_date", "key", "total_tracks","id", "duration_ms", "explicit", "mode"],  axis="columns") 
data.head()

playlistsongs = get_tracks()
playlistsongs


dance1=playlistsongs[0][1][0]['danceability']  
ave1=playlistsongs[0][1][0]['energy']  
dance2=playlistsongs[1][1][0]['danceability']
ave2=playlistsongs[1][1][0]['energy']  
dance3=playlistsongs[2][1][0]['danceability']
ave3=playlistsongs[2][1][0]['energy']  
dance4=playlistsongs[3][1][0]['danceability']
ave4=playlistsongs[3][1][0]['energy']  
dance5=playlistsongs[4][1][0]['danceability']
ave5=playlistsongs[4][1][0]['energy']  

loud1=playlistsongs[0][1][0]['loudness'] 
loud2= playlistsongs[1][1][0]['loudness'] 
loud3=playlistsongs[2][1][0]['loudness'] 
loud4=playlistsongs[3][1][0]['loudness'] 
loud5=playlistsongs[4][1][0]['loudness'] 

ins1=playlistsongs[0][1][0]['instrumentalness']
ins2=playlistsongs[1][1][0]['instrumentalness']
ins3=playlistsongs[2][1][0]['instrumentalness']
ins4=playlistsongs[3][1][0]['instrumentalness']
ins5=playlistsongs[4][1][0]['instrumentalness']

val1=playlistsongs[0][1][0]['valence']
val2=playlistsongs[1][1][0]['valence']
val3=playlistsongs[2][1][0]['valence']
val4=playlistsongs[3][1][0]['valence']
val5=playlistsongs[4][1][0]['valence']

spe1=playlistsongs[0][1][0]['speechiness']
spe2=playlistsongs[1][1][0]['speechiness']
spe3=playlistsongs[2][1][0]['speechiness']
spe4=playlistsongs[3][1][0]['speechiness']
spe5=playlistsongs[4][1][0]['speechiness'] 

temp1 =playlistsongs[0][1][0]['tempo']
temp2=playlistsongs[1][1][0]['tempo']
temp3=playlistsongs[2][1][0]['tempo']
temp4=playlistsongs[3][1][0]['tempo']
temp5=playlistsongs[4][1][0]['tempo']


#scale de features die ik gebruik
data['tempo'] /= 380

data['loudness'] /=  60
averagedance = (dance1 + dance2 + dance3 + dance4 +dance5)/5  
averageval = (val1 + val2 + val3 +val4 +val5)/5
tempo1 = (temp1 +temp2 +temp3+temp4+temp5)/5
avetempo = tempo1/380
aveenergy = (ave1 + ave2+ ave3 +ave4 +ave5)/5 
speech = (spe1 + spe2+spe3+spe4+spe5)/5
instru = (ins1 + ins2+ ins3+ins4+ins5)/5 
loudnes = (loud1+loud2+loud4+loud5)/5 
loudness =loudnes/60 




#stappen voor recommendation: 
#Bereken afstand dmv pythagoras
#voeg afstand toe aan tabel
#vindt de kleinste afstand 
#geef een tabel met afstand en titel

#de methodes zijn verschillende combinaties met audio features. 
def calculation(pyt):
    distance = pyt**0.5
    data["distopoint"] = distance
    recommendation = data[data["distopoint"]==data["distopoint"].min()]
    result= recommendation[["Title","distopoint","Artist"]].copy() 
    if len(result)>1:
        result= result.sample()
    else: result
    return result

def method1():
    pyt = (averagedance - data["danceability"]  )**2 + ( averageval-data["valence"] )**2 + ( avetempo-data["tempo"] )**2 
    return calculation(pyt)

def method2():
    pyt = ( averagedance - data["danceability"] )**2 + ( averageval - data["valence"]  )**2 + (  aveenergy -data["energy"]  )**2
    return calculation(pyt)

def method3():
    pyt = ( averagedance - data["danceability"] )**2 + ( averageval - data["valence"]  )**2 + (  loudness -data["loudness"]  )**2
    return calculation(pyt)


def method4():
    pyt = (  data["speechiness"] -speech   )**2 + (  data["instrumentalness"] -instru )**2
    
    return calculation(pyt)

#random recommendation
def random():
    recommendation = data.sample()
    result= recommendation[["Title","Artist"]].copy() 
    return result 

een = method1()
twee = method2()
drie = method3()
vier=method4()
random=random()
allrec= pd.concat([een,twee,drie,vier,random])
allrec

,Title,distopoint,Artist
104,All Right Now,0.012148,Free
3607,No te apartes de mí,0.022833,Yahir
2050,Headstone,0.006133,Flatbush Zombies
179,Don't Look Back into the Sun,0.001028,The Libertines
4958,Timber (feat. Ke$ha),NaN,Pitbull


In [2]:
#Deze cell bevat het opvragen van data van spotify api

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#inloggen bij de spotify API en playlist link
CLIENT_ID = '82c974c12dd94f52a53884b52640c3c0'
CLIENT_SECRET = "9ea825d8d4b04e9e822cb845cade61a0"
PLAYLIST_LINK = "https://open.spotify.com/playlist/3VJlwgnV4IaxGK8uIEZMjV?si=ca8c506dd5d04663"

CLIENT_CREDENTIALS_MANAGER = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)
SP = spotipy.Spotify(client_credentials_manager=CLIENT_CREDENTIALS_MANAGER)

#get playlist
def get_playlist_uri(playlist_link):
    return playlist_link.split("/")[-1].split("?")[0]

#nummers + gegevens uit de playlist halen
def get_tracks():
    tracks = []
    playlist_uri = get_playlist_uri(PLAYLIST_LINK)
    for track in SP.playlist_tracks(playlist_uri)["items"]:
        track_uri = track["track"]["uri"]
        track_name = track["track"]["name"]
        result = track_name, SP.audio_features(track_uri)
        tracks.append(result)

    return tracks

    
    
playlistsongs = get_tracks()
playlistsongs

[('Hyperventilatie',
  [{'danceability': 0.738,
    'energy': 0.692,
    'key': 5,
    'loudness': -7.934,
    'mode': 1,
    'speechiness': 0.0502,
    'acousticness': 0.546,
    'instrumentalness': 0.00715,
    'liveness': 0.231,
    'valence': 0.593,
    'tempo': 112.979,
    'type': 'audio_features',
    'id': '3yHt2DUM0AhAs1olpPcm7s',
    'uri': 'spotify:track:3yHt2DUM0AhAs1olpPcm7s',
    'track_href': 'https://api.spotify.com/v1/tracks/3yHt2DUM0AhAs1olpPcm7s',
    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3yHt2DUM0AhAs1olpPcm7s',
    'duration_ms': 138654,
    'time_signature': 4}]),
 ('Last Christmas',
  [{'danceability': 0.735,
    'energy': 0.478,
    'key': 2,
    'loudness': -12.472,
    'mode': 1,
    'speechiness': 0.0293,
    'acousticness': 0.189,
    'instrumentalness': 2.48e-06,
    'liveness': 0.355,
    'valence': 0.947,
    'tempo': 107.682,
    'type': 'audio_features',
    'id': '2FRnf9qhLbvw8fu4IBXx78',
    'uri': 'spotify:track:2FRnf9qhLbvw8fu4I